<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/2_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 繪圖

- x_index_half 不會有最後一個 n+0.5（避免超出範圍）
- x 軸仍然顯示原本的 x_labels（但計算用數字）
- 灰色背景範圍從 x=0 到 x=0.5（對應 15:00:00 前的部分）
- 未來擴展時仍自動適應數據長度

In [ ]:
import plotly.graph_objects as go

# 原始數據
x_labels = ["20250202_140001 FAH55", "20250202_150201 FAH56", "20250202_171101 PAH52"]
y_values = [100, 150, 130]

# 動態生成 x 軸索引
x_index = list(range(len(x_labels)))  # 產生 [0, 1, 2, ..., n]
x_index_half = [i + 0.5 for i in x_index[:-1]]  # 產生 [0.5, 1.5, ..., n-0.5] (不包含最後一個)

fig = go.Figure()

# 添加折線圖，並設定 hovertext
fig.add_trace(go.Scatter(
    x=x_index,
    y=y_values,
    mode='lines+markers',
    name='數據',
    hovertext=x_labels,  # 懸停時顯示原始 x_labels
    hoverinfo="text+y"   # 只顯示 hovertext (x_labels) + y 值
))

# 添加灰色背景 (最早的 x 位置到 +0.5)
fig.add_vrect(
    x0=x_index[0], x1=x_index[0] + 0.5,  # 只標示第一個點到 0.5
    fillcolor="gray", opacity=0.3, layer="below", line_width=0
)

# 設定 x 軸標籤
fig.update_layout(
    title="自動生成 X 軸索引並保持原始標籤",
    xaxis_title="時間+編號",
    yaxis_title="數值",
    xaxis=dict(
        tickvals=x_index,      # 設定 x 軸的索引
        ticktext=x_labels,     # 替換成原始時間+編號
        range=[-0.5, len(x_labels) - 1]  # 確保標籤不擠在邊界
    )
)

fig.show()


## 2. func

1. 數字化 x 軸
把 x_col 轉成 數字索引 (0, 1, 2, ...)，但 x 軸仍顯示原本的時間+編號。
判斷哪些點需要灰底
2. 解析 x_col 的時間部分，如果 < start_time，則該點的 .5 索引 會加灰底。
3. 畫折線圖 & 加入灰色區間。
4. 自動適應不同數據長度。

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_time_series_with_shading(df, x_col, y_col, start_time):
    """
    繪製折線圖，並對 x_col 時間部分小於 start_time 的 0.5 索引點加上灰底。

    參數：
    - df: pandas DataFrame，包含 x_col 和 y_col
    - x_col: str，表示 x 軸的欄位名稱（格式："YYYYMMDD_HHMMSS 編號"）
    - y_col: str，表示 y 軸的欄位名稱（數值型）
    - start_time: str，格式為 "HHMMSS"，用來判別灰底區域

    回傳：
    - plotly Figure 物件
    """

    # 1️⃣ 生成 x 軸數字索引
    df = df.copy()
    df["x_index"] = range(len(df))  # 生成 0, 1, 2, ...

    # 2️⃣ 解析時間部分，用來判斷灰色區間
    df["time_part"] = df[x_col].str.split("_").str[1]  # 提取時間部分 (HHMMSS)

    # 3️⃣ 找出需要加灰底的 x_index_half
    x_index_half = [
        i + 0.5 for i, time_str in zip(df["x_index"], df["time_part"])
        if time_str < start_time
    ]

    # 4️⃣ 建立圖表
    fig = go.Figure()

    # 添加折線圖
    fig.add_trace(go.Scatter(
        x=df["x_index"],
        y=df[y_col],
        mode='lines+markers',
        name='數據'
    ))

    # 5️⃣ 加入灰底區間 (多個範圍)
    for x_half in x_index_half:
        fig.add_vrect(
            x0=x_half - 0.5, x1=x_half,  # 標記區間 (e.g., 0 ~ 0.5)
            fillcolor="gray", opacity=0.3, layer="below", line_width=0
        )

    # 6️⃣ 設定 x 軸標籤
    fig.update_layout(
        title="時間序列折線圖 (帶灰色區間)",
        xaxis_title=x_col,
        yaxis_title=y_col,
        xaxis=dict(
            tickvals=df["x_index"],  # 數字索引
            ticktext=df[x_col],      # 保持原本 x 軸顯示
            range=[-0.5, len(df) - 0.5]  # 確保不擠在邊界
        )
    )

    return fig


1. x_index 仍為整數索引（0, 1, 2, ...）。
2. 灰底範圍用 x_index_half 計算：
3. 當 time_part < start_time，對應的 x_index 產生 灰色範圍 x0 = i, x1 = i + 0.5。
4. 畫圖時加 add_vrect(x0=i, x1=i+0.5) 來繪製灰色背景。

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def plot_time_series_with_shading(df, x_col, y_col, start_time):
    """
    繪製折線圖，並對 x_col 時間部分小於 start_time 的 0.5 索引點加上灰底。

    參數：
    - df: pandas DataFrame，包含 x_col 和 y_col
    - x_col: str，表示 x 軸的欄位名稱（格式："YYYYMMDD_HHMMSS 編號"）
    - y_col: str，表示 y 軸的欄位名稱（數值型）
    - start_time: str，格式為 "HHMMSS"，用來判別灰底區域

    回傳：
    - plotly Figure 物件
    """

    # 1️⃣ 生成 x 軸數字索引
    df = df.copy()
    df["x_index"] = range(len(df))  # 生成 [0, 1, 2, ..., n]

    # 2️⃣ 解析時間部分，用來判斷灰色區間
    df["time_part"] = df[x_col].str.split("_").str[1]  # 提取時間部分 (HHMMSS)

    # 3️⃣ 找出需要加灰底的區間
    shade_intervals = [
        (i, i + 0.5) for i, time_str in zip(df["x_index"], df["time_part"])
        if time_str < start_time
    ]

    # 4️⃣ 建立圖表
    fig = go.Figure()

    # 添加折線圖
    fig.add_trace(go.Scatter(
        x=df["x_index"],
        y=df[y_col],
        mode='lines+markers',
        name='數據'
    ))

    # 5️⃣ 加入灰底區間 (多個範圍)
    for x0, x1 in shade_intervals:
        fig.add_vrect(
            x0=x0, x1=x1,  # 標記區間 (e.g., 0 ~ 0.5)
            fillcolor="gray", opacity=0.3, layer="below", line_width=0
        )

    # 6️⃣ 設定 x 軸標籤
    fig.update_layout(
        title="時間序列折線圖 (帶灰色區間)",
        xaxis_title=x_col,
        yaxis_title=y_col,
        xaxis=dict(
            tickvals=df["x_index"],  # 數字索引
            ticktext=df[x_col],      # 保持原本 x 軸顯示
            range=[-0.5, len(df) - 0.5]  # 確保不擠在邊界
        )
    )

    return fig
